In [1]:
# === PRIME FAMILY φ-PHASE TEST (A’) — families + triads + Gaussian visuals ===
# No installs; runs on stock Colab. Keeps runtime light with caps and simple nulls.

import os, math, json, csv, numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng

# ---------------- Setup ----------------
rng = default_rng(137)
plt.rcParams["figure.dpi"] = 120
BASE_DIR = "prime_family_phi_test"
os.makedirs(BASE_DIR, exist_ok=True)

# Windows (disjoint; moderate) and caps
WINDOWS = [(10**6, 10**7), (10**7, 5*10**7)]     # two ranges
CAP_PRIMES = 250_000                             # cap primes per window (speed)
CAP_GAUSS  = 450                                 # search box half-width for Gaussian primes (|a|,|b|<=CAP_GAUSS)

phi = (1 + 5**0.5) / 2
NEAR_PHI = [1.60, 1.61, 1.62, 1.63]
BASES = [phi] + NEAR_PHI

# ---------------- Utilities ----------------
def primes_in_window(lo, hi, cap=None):
    from sympy import primerange
    arr = np.fromiter(primerange(int(lo), int(hi)), dtype=np.int64)
    if cap and arr.size > cap:
        idx = rng.choice(arr.size, size=cap, replace=False)
        arr = arr[idx]
    return np.sort(arr)

def emd_hist_01(phases, bins=200):
    x = np.asarray(phases, dtype=float) % 1.0
    h, edges = np.histogram(x, bins=bins, range=(0,1), density=True)
    centers = (edges[:-1] + edges[1:]) / 2
    cdf = np.cumsum(h)
    if cdf[-1]==0: return 0.0
    cdf /= cdf[-1]
    cdf_u = centers  # CDF of U[0,1]
    return float(np.trapezoid(np.abs(cdf - cdf_u), centers))

def kuiper_uniform01(x, m_mc=1000):
    x = np.sort(np.asarray(x, dtype=float) % 1.0)
    n = x.size
    if n==0: return 0.0, 1.0
    grid = np.arange(1, n+1)/n
    Dp = np.max(grid - x)
    Dm = np.max(x - (np.arange(0, n)/n))
    V  = float(Dp + Dm)
    u  = np.sort(rng.random((m_mc, n)), axis=1)
    Dp0 = np.max(grid - u, axis=1)
    u2  = np.sort(rng.random((m_mc, n)), axis=1)  # independent for stability
    Dm0 = np.max(u2 - (np.arange(0, n)/n), axis=1)
    V0  = Dp0 + Dm0
    p   = (np.sum(V0 >= V) + 1) / (m_mc + 1)
    return V, float(p)

def phi_phase(vals, base):
    v = np.asarray(vals, dtype=float)
    # guard small/negatives:
    v = v[(v>0) & np.isfinite(v)]
    if v.size==0: return np.array([])
    return (np.log(v)/np.log(base)) % 1.0

def family_pairs(primes, d):
    pset = set(primes.tolist())
    out  = [(p, p+d) for p in primes if (p+d) in pset]
    return np.array(out, dtype=np.int64)

def interfamily_gaps(base_primes):
    # gaps between successive BASES of the family (not the inside-pair gap which is fixed d)
    bp = np.unique(base_primes)
    if bp.size < 3: return np.array([])
    diffs = np.diff(bp)
    return diffs / np.log(bp[:-1])

def triad_layers(phases, k):
    # return phases split by index mod k
    layers = []
    for r in range(k):
        layers.append(phases[r::k])
    return layers

# Gaussian primes (strict interior: a,b != 0; a^2+b^2 is rational prime)
def gaussian_primes_box(M):
    from sympy import isprime
    pts = []
    for a in range(-M, M+1):
        if a==0: continue
        for b in range(-M, M+1):
            if b==0: continue
            r2 = a*a + b*b
            if isprime(r2):
                pts.append((a,b))
    return np.array(pts, dtype=np.int64)

# ---------------- Main analysis ----------------
summary_rows = []
triad_rows   = []
gauss_rows   = []

FAMILIES = [
    ("twin",   2),
    ("cousin", 4),
    ("sexy",   6),
]

for (lo, hi) in WINDOWS:
    primes = primes_in_window(lo, hi, cap=CAP_PRIMES)

    # Build families
    fam_data = {}
    for name, d in FAMILIES:
        pairs = family_pairs(primes, d)
        base_ps = pairs[:,0] if pairs.size else np.array([], dtype=np.int64)
        gaps = interfamily_gaps(base_ps)  # sequence to analyze
        fam_data[name] = dict(pairs=pairs, base_primes=base_ps, gaps=gaps)

    # For each family: φ-phase of interfamily gaps under each base; stats + plot
    for name in fam_data:
        gaps = fam_data[name]["gaps"]
        if gaps.size == 0:
            # record empty
            for b in BASES:
                summary_rows.append([lo, hi, name, b, 0, 0.0, 1.0, 0.0])
            continue

        emd_list, p_list = [], []
        plt.figure()
        for b in BASES:
            phases = phi_phase(gaps, b)
            V, p = kuiper_uniform01(phases, m_mc=1200)
            emd = emd_hist_01(phases, bins=160)
            emd_list.append((b, emd))
            p_list.append((b, p))
            # quick density line
            h, edges = np.histogram(phases, bins=60, range=(0,1), density=True)
            centers = 0.5*(edges[:-1]+edges[1:])
            plt.plot(centers, h, label=f"b={b:.4f}")

            summary_rows.append([lo, hi, name, float(b), phases.size, float(emd), float(p), float(V)])

        plt.title(f"{name} — φ-phase densities (window [{lo},{hi}))")
        plt.xlabel("{log_gap}_b mod 1"); plt.ylabel("density"); plt.legend(fontsize=8, ncol=2)
        os.makedirs(f"{BASE_DIR}/plots", exist_ok=True)
        plt.tight_layout()
        plt.savefig(f"{BASE_DIR}/plots/{name}_density_{lo}_{hi}.png"); plt.close()

        # Triad layers for φ only
        phases_phi = phi_phase(gaps, phi)
        for k in [3,6,9]:
            layers = triad_layers(phases_phi, k)
            # EMD per layer vs uniform
            for j,layer in enumerate(layers, start=1):
                emd_k = emd_hist_01(layer, bins=80) if layer.size>0 else 0.0
                triad_rows.append([lo, hi, name, k, j, layer.size, emd_k])

            # plot layers together
            plt.figure()
            for j,layer in enumerate(layers, start=1):
                if layer.size==0: continue
                h, edges = np.histogram(layer, bins=60, range=(0,1), density=True)
                centers = 0.5*(edges[:-1]+edges[1:])
                plt.plot(centers, h, label=f"mod{k} layer {j}")
            plt.title(f"{name} — φ-phase layers mod {k} (window [{lo},{hi}))")
            plt.xlabel("{log_gap}_φ mod 1"); plt.ylabel("density"); plt.legend(fontsize=8, ncol=2)
            plt.tight_layout()
            plt.savefig(f"{BASE_DIR}/plots/{name}_layers_mod{k}_{lo}_{hi}.png"); plt.close()

    # Gaussian primes visuals (two “cymatic” colourings)
    M = CAP_GAUSS
    pts = gaussian_primes_box(M)
    if pts.size:
        a = pts[:,0].astype(float); b = pts[:,1].astype(float)
        r = np.sqrt(a*a + b*b)
        theta = np.arctan2(b, a) % (2*np.pi)
        # φ-phase of radius; φ-phase of angle normalized to [0,1)
        r_phase = (np.log(r+1e-12)/np.log(phi)) % 1.0
        th_phase = (theta/(2*np.pi))  # already in [0,1)

        # store few stats
        emd_r  = emd_hist_01(r_phase, bins=160)
        emd_th = emd_hist_01(th_phase, bins=160)
        gauss_rows.append([lo, hi, pts.shape[0], float(emd_r), float(emd_th)])

        # plot radius-phase colouring
        plt.figure(figsize=(6,6))
        plt.scatter(a, b, c=r_phase, s=2, cmap="hsv")
        plt.gca().set_aspect("equal", adjustable="box")
        plt.title(f"Gaussian primes (|a|,|b|≤{M}) — colour=r φ-phase")
        plt.tight_layout()
        plt.savefig(f"{BASE_DIR}/plots/gauss_rphase_{lo}_{hi}.png"); plt.close()

        # plot angle-phase colouring
        plt.figure(figsize=(6,6))
        plt.scatter(a, b, c=th_phase, s=2, cmap="hsv")
        plt.gca().set_aspect("equal", adjustable="box")
        plt.title(f"Gaussian primes (|a|,|b|≤{M}) — colour=angle/(2π)")
        plt.tight_layout()
        plt.savefig(f"{BASE_DIR}/plots/gauss_angle_{lo}_{hi}.png"); plt.close()

# ---------------- Save summaries + verdict ----------------
os.makedirs(f"{BASE_DIR}/artifacts", exist_ok=True)

with open(f"{BASE_DIR}/artifacts/family_summary.csv","w") as f:
    w = csv.writer(f)
    w.writerow(["lo","hi","family","base","n_phases","emd","kuiper_p","kuiper_V"])
    w.writerows(summary_rows)

with open(f"{BASE_DIR}/artifacts/triad_layers.csv","w") as f:
    w = csv.writer(f)
    w.writerow(["lo","hi","family","mod_k","layer_idx","n","emd"])
    w.writerows(triad_rows)

with open(f"{BASE_DIR}/artifacts/gaussian_summary.csv","w") as f:
    w = csv.writer(f)
    w.writerow(["lo","hi","n_points","emd_r_phase","emd_angle"])
    w.writerows(gauss_rows)

# Decision per preregistered bar
report = {"windows": WINDOWS, "families": [], "overall_phi_special": False}

for (lo,hi) in WINDOWS:
    window_families = []
    for name,_ in FAMILIES:
        rows = [r for r in summary_rows if r[0]==lo and r[1]==hi and r[2]==name]
        if not rows:
            window_families.append({"family":name,"pass":False,"why":"no data"});
            continue
        # φ metrics
        row_phi = [r for r in rows if abs(r[3]-phi)<1e-12][0]
        emd_phi = row_phi[5]; p_phi = row_phi[6]
        emd_map = {r[3]: r[5] for r in rows}
        phi_is_max = (max(emd_map, key=emd_map.get) == phi)
        # triad consistency (any of mod 3/6/9: φ layers nonflat and consistent)
        tri = [r for r in triad_rows if r[0]==lo and r[1]==hi and r[2]==name]
        tri_pass = False
        for k in [3,6,9]:
            vals = [r for r in tri if r[3]==k]
            if not vals: continue
            # basic nonflatness: max layer emd >= 1.15 * median layer emd
            layer_emds = [r[6] for r in vals]
            if len(layer_emds)>=2 and (max(layer_emds) >= 1.15*np.median(layer_emds)):
                tri_pass = True; break

        fam_pass = (phi_is_max and (p_phi <= 0.01) and tri_pass)
        window_families.append({
            "family": name,
            "phi_is_max": phi_is_max,
            "phi_emd": emd_phi,
            "phi_kuiper_p": p_phi,
            "triad_ok": tri_pass,
            "pass": fam_pass
        })
    report["families"].append({"window":[lo,hi], "results":window_families})

# overall: ≥2 families pass in both windows
def count_pass(res): return sum(1 for x in res if x["pass"])
both = all(count_pass(win["results"]) >= 2 for win in report["families"])
report["overall_phi_special"] = bool(both)

with open(f"{BASE_DIR}/artifacts/family_report.json","w") as f:
    json.dump(report, f, indent=2)

print(json.dumps(report, indent=2))
print(f"\nArtifacts in {BASE_DIR}/artifacts ; Plots in {BASE_DIR}/plots")


{
  "windows": [
    [
      1000000,
      10000000
    ],
    [
      10000000,
      50000000
    ]
  ],
  "families": [
    {
      "window": [
        1000000,
        10000000
      ],
      "results": [
        {
          "family": "twin",
          "phi_is_max": false,
          "phi_emd": 0.003400386794508705,
          "phi_kuiper_p": 0.12239800166527894,
          "triad_ok": true,
          "pass": false
        },
        {
          "family": "cousin",
          "phi_is_max": false,
          "phi_emd": 0.0026753646060078792,
          "phi_kuiper_p": 0.34388009991673607,
          "triad_ok": true,
          "pass": false
        },
        {
          "family": "sexy",
          "phi_is_max": false,
          "phi_emd": 0.015029624235337893,
          "phi_kuiper_p": 0.0008326394671107411,
          "triad_ok": true,
          "pass": false
        }
      ]
    },
    {
      "window": [
        10000000,
        50000000
      ],
      "results": [
        {
        